In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


# Step 1: 사용자가 입력한 장르에 따라 영화 1편 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template(
    """
    {movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3문장으로 요약해 주세요.
    [출력 형식]
    제목: {movie}
    줄거리: 
    (첫 번째 문장)
    (두 번째 문장)
    (세 번째 문장)
    """
    )

# OpenAI 모델 사용
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "코미디"})
    print(response)  
except Exception as e:
    print(f"에러 발생 : {e}")

제목: 코미디 장르의 영화 중 '슈퍼스타'(2021)를 추천해 드립니다.

줄거리: 
 글로벌 스타가 되고 싶은 트로트 가수 '극동'과 열혈 팬인 인플루언서 '아리'가 만나 벌어지는 이야기를 그린 코믹한 영화입니다.
 영화는 웃음과 감동을 적절하게 배합하여 관객들에게 즐거운 경험을 제공합니다.
 두 사람의 만남으로 인해 예상치 못한 상황들이 벌어지고, 코믹한 상황들이 연출됩니다.
